# مهمة تصنيف النصوص

كما ذكرنا، سنركز على مهمة تصنيف النصوص البسيطة باستخدام مجموعة بيانات **AG_NEWS**، والتي تهدف إلى تصنيف عناوين الأخبار إلى واحدة من 4 فئات: العالم، الرياضة، الأعمال، والعلوم/التكنولوجيا.

## مجموعة البيانات

تم تضمين هذه المجموعة في وحدة [`torchtext`](https://github.com/pytorch/text)، مما يجعل الوصول إليها سهلاً.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

هنا، يحتوي `train_dataset` و `test_dataset` على مجموعات تُرجع أزواجًا من التصنيف (رقم الفئة) والنص على التوالي، على سبيل المثال:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

لذا، دعونا نطبع أول 10 عناوين جديدة من مجموعتنا البيانية:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

لأن مجموعات البيانات هي مكررات، إذا أردنا استخدام البيانات عدة مرات، نحتاج إلى تحويلها إلى قائمة:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## تقسيم النص إلى وحدات

الآن نحتاج إلى تحويل النص إلى **أرقام** يمكن تمثيلها كـ **موترات**. إذا أردنا تمثيل النص على مستوى الكلمات، يجب علينا القيام بأمرين:
* استخدام **المقسم النصي** لتقسيم النص إلى **وحدات نصية**
* بناء **قاموس** لهذه الوحدات النصية.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

باستخدام المفردات، يمكننا بسهولة ترميز السلسلة المرمزة إلى مجموعة من الأرقام:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## تمثيل النص باستخدام حقيبة الكلمات

لأن الكلمات تمثل المعاني، يمكننا أحيانًا فهم معنى النص بمجرد النظر إلى الكلمات الفردية، بغض النظر عن ترتيبها في الجملة. على سبيل المثال، عند تصنيف الأخبار، كلمات مثل *الطقس* و*الثلج* من المحتمل أن تشير إلى *توقعات الطقس*، بينما كلمات مثل *الأسهم* و*الدولار* قد تشير إلى *الأخبار المالية*.

**حقيبة الكلمات** (BoW) هي الطريقة الأكثر استخدامًا لتمثيل النصوص بشكل تقليدي باستخدام المتجهات. يتم ربط كل كلمة بمؤشر في المتجه، ويحتوي عنصر المتجه على عدد مرات ظهور الكلمة في مستند معين.

![صورة توضح كيفية تمثيل حقيبة الكلمات باستخدام المتجهات في الذاكرة.](../../../../../translated_images/ar/bag-of-words-example.606fc1738f1d7ba9.webp)

> **ملاحظة**: يمكنك أيضًا التفكير في حقيبة الكلمات كجمع لجميع المتجهات المشفرة بطريقة "واحد-ساخن" للكلمات الفردية في النص.

فيما يلي مثال على كيفية إنشاء تمثيل حقيبة الكلمات باستخدام مكتبة Scikit Learn بلغة بايثون:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

لحساب متجه حقيبة الكلمات من تمثيل المتجه لمجموعة بيانات AG_NEWS الخاصة بنا، يمكننا استخدام الدالة التالية:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **ملاحظة:** هنا نستخدم المتغير العالمي `vocab_size` لتحديد الحجم الافتراضي للمفردات. نظرًا لأن حجم المفردات غالبًا ما يكون كبيرًا، يمكننا تحديد حجم المفردات بالكلمات الأكثر شيوعًا. حاول تقليل قيمة `vocab_size` وتشغيل الكود أدناه، وشاهد كيف يؤثر ذلك على الدقة. يجب أن تتوقع انخفاضًا في الدقة، ولكن ليس بشكل كبير، مقابل أداء أعلى.


## تدريب مصنف BoW

الآن بعد أن تعلمنا كيفية بناء تمثيل حقيبة الكلمات (Bag-of-Words) لنصوصنا، دعونا ندرب مصنفًا يعتمد عليه. أولاً، نحتاج إلى تحويل مجموعة البيانات الخاصة بنا للتدريب بطريقة يتم فيها تحويل جميع تمثيلات المتجهات الموضعية إلى تمثيل حقيبة الكلمات. يمكن تحقيق ذلك عن طريق تمرير وظيفة `bowify` كمعامل `collate_fn` إلى `DataLoader` القياسي في مكتبة torch:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

الآن دعونا نعرّف شبكة عصبية تصنيفية بسيطة تحتوي على طبقة خطية واحدة. حجم متجه الإدخال يساوي `vocab_size`، وحجم الإخراج يتوافق مع عدد الفئات (4). نظرًا لأننا نقوم بحل مهمة التصنيف، فإن وظيفة التنشيط النهائية هي `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

الآن سنقوم بتعريف حلقة التدريب القياسية في PyTorch. نظرًا لأن مجموعة البيانات لدينا كبيرة جدًا، ولأغراض التعليم، سنقوم بالتدريب فقط لمدة حقبة واحدة، وأحيانًا حتى أقل من حقبة واحدة (يسمح لنا تحديد معامل `epoch_size` بتقييد التدريب). سنقوم أيضًا بالإبلاغ عن دقة التدريب المتراكمة أثناء التدريب؛ يتم تحديد تكرار الإبلاغ باستخدام معامل `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## ثنائيات الكلمات، ثلاثيات الكلمات، و N-Grams

أحد قيود طريقة حقيبة الكلمات هو أن بعض الكلمات تكون جزءًا من تعبيرات متعددة الكلمات. على سبيل المثال، الكلمة "hot dog" لها معنى مختلف تمامًا عن الكلمتين "hot" و"dog" في سياقات أخرى. إذا قمنا دائمًا بتمثيل الكلمات "hot" و"dog" بنفس المتجهات، فقد يؤدي ذلك إلى إرباك النموذج الخاص بنا.

لمعالجة هذا الأمر، يتم استخدام **تمثيلات N-gram** غالبًا في طرق تصنيف المستندات، حيث يكون تكرار كل كلمة، أو ثنائية الكلمات، أو ثلاثية الكلمات ميزة مفيدة لتدريب المصنفات. في تمثيل ثنائيات الكلمات، على سبيل المثال، سنضيف جميع أزواج الكلمات إلى المفردات، بالإضافة إلى الكلمات الأصلية.

فيما يلي مثال على كيفية إنشاء تمثيل حقيبة كلمات لثنائيات الكلمات باستخدام مكتبة Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

العيب الرئيسي في نهج N-gram هو أن حجم المفردات يبدأ في النمو بسرعة كبيرة. في الواقع، نحتاج إلى دمج تمثيل N-gram مع بعض تقنيات تقليل الأبعاد، مثل *التضمينات*، والتي سنناقشها في الوحدة التالية.

لاستخدام تمثيل N-gram في مجموعة بيانات **AG News** الخاصة بنا، نحتاج إلى بناء مفردات ngram خاصة:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


يمكننا استخدام نفس الكود أعلاه لتدريب المصنف، ولكن ذلك سيكون غير فعال من حيث استهلاك الذاكرة. في الوحدة التالية، سنقوم بتدريب مصنف ثنائي باستخدام التضمينات.

> **ملاحظة:** يمكنك فقط الاحتفاظ بتلك ngrams التي تظهر في النص أكثر من عدد مرات محدد. هذا سيضمن استبعاد الثنائيات النادرة، وسيقلل بشكل كبير من الأبعاد. للقيام بذلك، قم بتعيين قيمة أعلى للمعامل `min_freq`، وراقب كيف يتغير طول المفردات.


## تكرار المصطلح وتكرار الوثيقة العكسي TF-IDF

في تمثيل BoW، يتم وزن ظهور الكلمات بالتساوي بغض النظر عن الكلمة نفسها. ومع ذلك، من الواضح أن الكلمات الشائعة مثل *a* و*in* أقل أهمية بكثير للتصنيف مقارنة بالمصطلحات المتخصصة. في الواقع، في معظم مهام معالجة اللغة الطبيعية، تكون بعض الكلمات أكثر أهمية من غيرها.

**TF-IDF** تعني **تكرار المصطلح–تكرار الوثيقة العكسي**. وهي عبارة عن تعديل لطريقة حقيبة الكلمات، حيث يتم استخدام قيمة عائمة بدلاً من القيمة الثنائية 0/1 التي تشير إلى ظهور كلمة في وثيقة، وهذه القيمة ترتبط بتكرار ظهور الكلمة في النصوص.

بشكل أكثر رسمية، يتم تعريف الوزن $w_{ij}$ لكلمة $i$ في الوثيقة $j$ على النحو التالي:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
حيث:
* $tf_{ij}$ هو عدد مرات ظهور الكلمة $i$ في الوثيقة $j$، أي القيمة التي رأيناها في طريقة BoW
* $N$ هو عدد الوثائق في المجموعة
* $df_i$ هو عدد الوثائق التي تحتوي على الكلمة $i$ في المجموعة بأكملها

قيمة TF-IDF $w_{ij}$ تزداد بشكل يتناسب مع عدد مرات ظهور الكلمة في الوثيقة، ويتم تعديلها بناءً على عدد الوثائق في النصوص التي تحتوي على الكلمة، مما يساعد على التكيف مع حقيقة أن بعض الكلمات تظهر بشكل أكثر تكرارًا من غيرها. على سبيل المثال، إذا ظهرت الكلمة في *كل* الوثائق في المجموعة، فإن $df_i=N$، و$w_{ij}=0$، وسيتم تجاهل تلك المصطلحات تمامًا.

يمكنك بسهولة إنشاء تمثيل TF-IDF للنص باستخدام مكتبة Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## الخاتمة

على الرغم من أن تمثيلات TF-IDF توفر وزنًا تكراريًا للكلمات المختلفة، إلا أنها غير قادرة على تمثيل المعنى أو الترتيب. كما قال اللغوي الشهير ج. ر. فيرث في عام 1935: "المعنى الكامل للكلمة دائمًا ما يكون سياقيًا، ولا يمكن أخذ أي دراسة للمعنى بعيدًا عن السياق على محمل الجد." سنتعلم لاحقًا في الدورة كيفية التقاط المعلومات السياقية من النص باستخدام نمذجة اللغة.



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
